In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from sklearn.impute import SimpleImputer

df = pd.read_csv('./dataset/in-vehicle-coupon-recommendation.csv')
df.head()

,destination,passenger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passenger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

In [3]:
df.nunique()

destination              3
passenger                4
weather                  3
temperature              3
time                     5
coupon                   5
expiration               2
gender                   2
age                      8
maritalStatus            5
has_children             2
education                6
occupation              25
income                   9
car                      5
Bar                      5
CoffeeHouse              5
CarryAway                5
RestaurantLessThan20     5
Restaurant20To50         5
toCoupon_GEQ5min         1
toCoupon_GEQ15min        2
toCoupon_GEQ25min        2
direction_same           2
direction_opp            2
Y                        2
dtype: int64

In [4]:
df.skew()

temperature         -0.643390
has_children         0.348644
toCoupon_GEQ5min     0.000000
toCoupon_GEQ15min   -0.247890
toCoupon_GEQ25min    2.351805
direction_same       1.389366
direction_opp       -1.389366
Y                   -0.276364
dtype: float64

In [5]:
df.isnull().sum().sort_values(ascending=False) * 100 / len(df)

car                     99.148534
CoffeeHouse              1.710817
Restaurant20To50         1.490066
CarryAway                1.190476
RestaurantLessThan20     1.024913
Bar                      0.843582
Y                        0.000000
maritalStatus            0.000000
passenger                0.000000
weather                  0.000000
temperature              0.000000
time                     0.000000
coupon                   0.000000
expiration               0.000000
gender                   0.000000
age                      0.000000
occupation               0.000000
has_children             0.000000
education                0.000000
direction_opp            0.000000
income                   0.000000
toCoupon_GEQ5min         0.000000
toCoupon_GEQ15min        0.000000
toCoupon_GEQ25min        0.000000
direction_same           0.000000
destination              0.000000
dtype: float64

In [6]:
df['Y'].value_counts() / len(df)

1    0.568433
0    0.431567
Name: Y, dtype: float64

In [7]:
df.duplicated().sum()

74

In [8]:
df[df.duplicated()]

,destination,passenger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
4192,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,less1,less1,1,1,1,0,1,1
4236,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Male,26,Single,...,gt8,gt8,4~8,less1,1,1,1,0,1,1
4280,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,26,Single,...,never,4~8,1~3,less1,1,1,1,0,1,1
4324,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,46,Single,...,never,4~8,1~3,1~3,1,1,1,0,1,1
4409,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,21,Single,...,never,less1,1~3,never,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,Home,Alone,Sunny,80,6PM,Bar,2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,0,0,1,0,1
8512,Home,Partner,Sunny,30,10PM,Carry out & Take away,2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,0,0,1,1
8513,Work,Alone,Rainy,55,7AM,Restaurant(<20),2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,1,0,1,0
8515,Work,Alone,Snowy,30,7AM,Restaurant(20-50),1d,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,1,0,1,0


In [9]:
X, Y = df.iloc[:, :-1], df.iloc[:, -1]
print(X.columns)
print(X.shape, Y.shape)

Index(['destination', 'passenger', 'weather', 'temperature', 'time', 'coupon',
       'expiration', 'gender', 'age', 'maritalStatus', 'has_children',
       'education', 'occupation', 'income', 'car', 'Bar', 'CoffeeHouse',
       'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50',
       'toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min',
       'direction_same', 'direction_opp'],
      dtype='object')
(12684, 25) (12684,)


In [10]:
'''
    TODO:
        - Drop unnecessary columns:
            car, toCoupon_GE5min, direction_opp
        - Encode binary attrs:
            gender, has_children, direction
        - Encode ordinal attrs:
            + Frequency: Bar, CoffeeHouse, CarryAway, 
                         Cheap Restaurant, Expensive Restaurant
            + Others: temperature, time, expiration, age, education, 
                      income, toCoupon
        - Encode nominal attrs:
            destination, passenger, weather, coupon, maritalStatus, 
            occupation
'''

drop_attrs = ['car', 'toCoupon_GEQ5min', 'direction_opp']
binary_attrs = ['gender', 'has_children', 'direction_same']
frequency_attrs = ['Bar', 'CoffeeHouse', 'CarryAway', 
                   'RestaurantLessThan20', 'Restaurant20To50']
ordinal_attrs = ['temperature', 'time', 'expiration', 'age', 
                 'education', 'income', 'toCoupon']
nominal_attrs = ['destination', 'passenger', 'weather', 'coupon', 
                 'maritalStatus', 'occupation']

In [11]:
# drop columns 
X = X.drop(columns=drop_attrs)

In [12]:
# Encoding binary attributes
gender_encoder = LabelEncoder()
X['gender'] = gender_encoder.fit_transform(X['gender'])

X_binary = X[binary_attrs]

In [13]:
print(X_binary.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   gender          12684 non-null  int64
 1   has_children    12684 non-null  int64
 2   direction_same  12684 non-null  int64
dtypes: int64(3)
memory usage: 297.4 KB
None


In [14]:
# Encoding ordinal attributes
income_order = ['Less than $12500', '$12500 - $24999', 
                '$25000 - $37499', '$37500 - $49999', 
                '$50000 - $62499', '$62500 - $74999', 
                '$75000 - $87499', '$87500 - $99999', 
                '$100000 or More']
edu_order = ['Some High School', 'High School Graduate', 
             'Some college - no degree', 'Associates degree', 
             'Bachelors degree', 
             'Graduate degree (Masters or Doctorate)']
age_order = ['below21', '21', '26', '31', '36', '41', '46', '50plus']
time_order = ['7AM', '10AM', '2PM', '6PM', '10PM']
expiration_order = ['2h', '1d']


def reorder_encoding(attr, order):
    X[attr] = X[attr].astype('category')
    X[attr] = X[attr].cat.reorder_categories(
        new_categories=order,
        ordered=True
    )

    return pd.Series(X[attr].cat.codes, name=attr)


X_income = reorder_encoding('income', income_order)
X_edu = reorder_encoding('education', edu_order)
X_age = reorder_encoding('age', age_order)
X_time = reorder_encoding('time', time_order)
X_expiration = reorder_encoding('expiration', expiration_order)

In [15]:
freq_order = {
    'never': 0, 
    'less1': 1,
    '1~3': 2, 
    '4~8': 3, 
    'gt8': 4
}

for col in frequency_attrs:
    X[col] = X[col].map(freq_order, na_action='ignore')

In [16]:
'''
    Combine 2 attributes: toCoupon_GEQ15min, toCouponGEQ25min
        toCoupon_GEQ15min: whether it takes more than 15mins 
                           to get to the coupon (0/1)
        toCoupon_GEQ25min: whether it takes more thanh 25mins
                           to get to the coupon (0/1)
    After combining:
        toCoupon: (0, 1, 2)
            0: time to coupon < 15min
            1: 15 <= time to coupon < 25
            2: time to coupon >= 25
'''

to_coupon = X[['toCoupon_GEQ15min', 'toCoupon_GEQ25min']].to_numpy()
t = np.sum(to_coupon, axis=1)
X_toCoupon = pd.Series(
    data=np.sum(to_coupon, axis=1),
    name='toCoupon'
)

In [17]:
# Encoding nominal attributes
X_nominal = pd.get_dummies(X[nominal_attrs])

In [18]:
for t in [X_binary, X_income, X_edu, X_age, X_time, X_expiration,
     X[frequency_attrs], X_toCoupon, X_nominal, Y]:
    print(t.shape)

(12684, 3)
(12684,)
(12684,)
(12684,)
(12684,)
(12684,)
(12684, 5)
(12684,)
(12684, 45)
(12684,)


In [19]:
# sum up all encoded attributes
onehot_df = pd.concat(
    [X_binary, 
     X_income, 
     X_edu, 
     X_age, 
     X_time, 
     X_expiration,
     X[frequency_attrs], 
     X_toCoupon, 
     X_nominal, 
     Y],
    axis=1
)
print(onehot_df.shape)

(12684, 60)


In [20]:
onehot_df.duplicated().sum()

85

In [21]:
onehot_df.drop_duplicates(keep='first', inplace=True)

In [22]:
onehot_df.to_csv('./dataset/preprocessed.csv', index=False)